In [1]:
import findspark
findspark.init()
from pyspark import SparkConf,SparkContext
conf=SparkConf().setMaster("local").setAppName("Mi programa")
sc=SparkContext(conf=conf)
from pyspark.sql.types import StringType
from pyspark import SQLContext
sql_context=SQLContext(sc)
df_spark= sql_context.read.format("csv").option("header","true").option("inferSchema","true").load("base_datos_2008.csv")

In [2]:
df_spark=df_spark.sample(fraction=0.001,withReplacement=False)
df_spark=df_spark.withColumn("ArrDelay",df_spark["ArrDelay"].cast("integer"))

df2=df_spark.na.drop(subset=["ArrDelay","DepDelay","Distance"])
df2=df2.filter("ArrDelay is not NULL")
df2=df2.dropDuplicates()

In [3]:
df2.select("ArrDelay").filter("ArrDelay>60").take(5)

[Row(ArrDelay=73),
 Row(ArrDelay=77),
 Row(ArrDelay=129),
 Row(ArrDelay=216),
 Row(ArrDelay=571)]

In [4]:
#nos devuelve exactamente el valor 
df2.select("ArrDelay").filter("ArrDelay>60").take(5)[0]

Row(ArrDelay=73)

In [5]:
import numpy as np
media=np.mean(df2.select("ArrDelay").collect())

In [6]:
df2.select("ArrDelay").rdd.map(lambda x: (x-media)**2).take(10)

[array([23.33101313]),
 array([4117.76082625]),
 array([667.20020044]),
 array([719.86063793]),
 array([521.21888797]),
 array([354.577138]),
 array([219.93538804]),
 array([139.95407557]),
 array([33.99145062]),
 array([164.61451306])]

In [8]:
df2.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Ca

In [9]:
df2.groupBy("DayOfWeek").count().show()

+---------+-----+
|DayOfWeek|count|
+---------+-----+
|        1|  979|
|        6|  808|
|        3| 1050|
|        5| 1067|
|        4| 1005|
|        7|  993|
|        2| 1001|
+---------+-----+



In [10]:
df2.groupBy("DayOfWeek").mean("ArrDelay").show()

+---------+------------------+
|DayOfWeek|     avg(ArrDelay)|
+---------+------------------+
|        1| 9.231869254341165|
|        6|             5.375|
|        3| 7.791428571428572|
|        5|10.925960637300843|
|        4|10.080597014925374|
|        7| 9.342396777442096|
|        2|  8.31868131868132|
+---------+------------------+



In [11]:
df2.select("Origin").rdd.distinct().take(5)

[Row(Origin='PIT'),
 Row(Origin='FSM'),
 Row(Origin='SMF'),
 Row(Origin='CWA'),
 Row(Origin='AUS')]

In [12]:
df2.select("Origin").rdd.distinct().count()

237

In [13]:
df2.orderBy(df2.ArrDelay.desc()).take(5)

[Row(Year=2008, Month=10, DayofMonth=16, DayOfWeek=4, DepTime='716', CRSDepTime=845, ArrTime='1238', CRSArrTime=1412, UniqueCarrier='NW', FlightNum=394, TailNum='N353NW', ActualElapsedTime='202', CRSElapsedTime='207', AirTime='184', ArrDelay=1346, DepDelay='1351', Origin='SMF', Dest='MSP', Distance=1518, TaxiIn='6', TaxiOut='12', Cancelled=0, CancellationCode=None, Diverted=0, CarrierDelay='1346', WeatherDelay='0', NASDelay='0', SecurityDelay='0', LateAircraftDelay='0'),
 Row(Year=2008, Month=8, DayofMonth=10, DayOfWeek=7, DepTime='138', CRSDepTime=1545, ArrTime='241', CRSArrTime=1710, UniqueCarrier='WN', FlightNum=293, TailNum='N276WN', ActualElapsedTime='63', CRSElapsedTime='85', AirTime='45', ArrDelay=571, DepDelay='593', Origin='PVD', Dest='PHL', Distance=238, TaxiIn='5', TaxiOut='13', Cancelled=0, CancellationCode=None, Diverted=0, CarrierDelay='34', WeatherDelay='0', NASDelay='0', SecurityDelay='0', LateAircraftDelay='537'),
 Row(Year=2008, Month=6, DayofMonth=1, DayOfWeek=7, Dep